In [10]:
%%bq query
-- Find the weather stations located in Finland. 
--CREATE TABLE `bergqvist-sandbox.bike_demo.fi_gsod` AS
SELECT * FROM 
(
SELECT
   usaf,
   ST_GeogPoint(lon, lat) AS g,
   elev,
   name
FROM `bergqvist-sandbox.bike_demo.gsodstations` where country = "FI"
) fwban
join
(
SELECT
    stn, 
    year, mo, da,
    min,
    temp,
    max,
    visib,
    wdsp,
    mxpsd,
    gust,
    prcp,
    sndp,
    rain_drizzle,
    snow_ice_pellets,
    hail,
    thunder
FROM `bergqvist-sandbox.bike_demo.gsod2017`
) w17
ON fwban.usaf = w17.stn limit 10
;

usaf,g,elev,name,stn,year,mo,da,min,temp,max,visib,wdsp,mxpsd,gust,prcp,sndp,rain_drizzle,snow_ice_pellets,hail,thunder
027530,POINT(22.467 61.833),+0122.0,KANKAANPAA NIINISALO AIRFIELD,027530,2017,10,14,35.2,41.5,43.3,7.7,5.0,8.4,16.3,99.99,999.9,1,0,0,0
027260,POINT(23.583 68.6),+0374.0,ENONTEKIO NAKKALA,027260,2017,09,28,35.6,39.5,42.1,6.2,4.2,6.0,999.9,99.99,999.9,1,0,0,0
027680,POINT(26.05 61.2),+0092.0,HEINOLA ASEMANTAUS,027680,2017,11,30,32.2,33.1,35.1,4.3,999.9,999.9,999.9,99.99,999.9,1,1,0,0
028160,POINT(27.183 68.083),+0248.8,SODANKYLA VUOTSO,028160,2017,10,05,38.1,41.2,43.5,10.3,3.1,4.7,10.3,99.99,999.9,1,0,0,0
027380,POINT(25.7 63.733),+0153.3,PYHAJARVI OL OJAKYLA,027380,2017,10,04,43.5,47.0,51.1,16.1,6.3,9.5,19.2,99.99,999.9,1,0,0,0
027380,POINT(25.7 63.733),+0153.3,PYHAJARVI OL OJAKYLA,027380,2017,10,05,36.7,43.4,47.7,16.7,3.4,5.1,10.9,99.99,999.9,1,0,0,0
027380,POINT(25.7 63.733),+0153.3,PYHAJARVI OL OJAKYLA,027380,2017,09,21,41.5,45.4,48.0,10.4,6.9,8.5,15.9,99.99,999.9,1,0,0,0
028210,POINT(24.117 67.967),+0565.5,MUONIO SAMMALTUNTURI,028210,2017,09,22,37.2,42.2,44.1,0.1,15.1,18.3,26.0,99.99,999.9,1,0,0,0
028730,POINT(24.567 65.033),+0007.3,HAILUOTO MARJANIEMI,028730,2017,05,18,36.3,43.0,50.2,4.6,8.8,13.4,15.5,99.99,999.9,1,0,0,0
029650,POINT(25.633 60.967),+0078.7,LAHTI LAUNE,029650,2017,09,09,45.0,52.9,56.7,5.0,2.3,4.7,11.8,99.99,999.9,1,0,0,0


In [11]:
%%bq query
-- check out duplicated id:name pairs
with getmap as (
SELECT
name, stationid
FROM
`bergqvist-sandbox.bike_demo.hist` 
group by name, stationid
)
select 
c.*
from
(select stationid from 
getmap
group by stationid
having count(*) > 1 ) b
join getmap c
on b.stationid = c.stationid
;

name,stationid
test changement Kaivopuisto test changement,1
test Kaivopuisto test,1
Kaivopuisto,1
Teljäntie,100
Ulvilanpuisto,100
Puistokaari,55
Lahnalahdentie,55
Vanha Kauppahalli,47
Kauppatori,47


In [12]:
%%bq query
-- see what is going on with stationid 1 being used for test
SELECT
    name, count(*)
FROM
   `bergqvist-sandbox.bike_demo.hist` 
WHERE stationid = 1
GROUP BY name
;

name,f0_
Kaivopuisto,357405
test Kaivopuisto test,1
test changement Kaivopuisto test changement,1


In [13]:
%%bq query
-- check that location variation per id is "small" (some stations cross the street)
SELECT
   *,
   ST_GeogPoint(lon, lat) AS g
FROM
   `bergqvist-sandbox.bike_demo.hist` 
where not name LIKE "test%"
limit 100
;

stationId,name,timestamp,lon,lat,spacesAvailable,bikesAvailable,g
94,Laajalahden aukio,2017-08-17 18:04:01,24.8759851,60.1983016,2,24,POINT(24.8759851 60.1983016)
93,Torpanranta,2017-05-10 10:15:01,24.8713963,60.1948358,8,8,POINT(24.8713963 60.1948358)
587,Hauenkallio,2018-07-29 07:34:01,24.767831,60.162477,5,7,POINT(24.767831 60.162477)
106,Korppaanmäentie,2017-08-09 10:50:01,24.8989297,60.2034735,6,10,POINT(24.8989297 60.2034735)
78,Messeniuksenkatu,2018-05-31 06:46:01,24.9180102,60.1878352,19,8,POINT(24.9180102 60.1878352)
54,Gyldenintie,2018-06-13 08:25:01,24.8766658,60.1583842,13,17,POINT(24.8766658 60.1583842)
607,Nuottaniementie,2018-05-18 20:45:01,24.741546,60.147921,2,12,POINT(24.741546 60.147921)
749,Vallikatu,2018-07-10 10:09:01,24.813871,60.228463,1,0,POINT(24.813871 60.228463)
49,Annankatu,2018-07-23 07:09:01,24.9417708421,60.1634589429,14,1,POINT(24.94177084206 60.16345894291)
559,Pohjankulma,2018-06-07 02:02:01,24.794014,60.18288,3,28,POINT(24.794014 60.18288)


In [14]:
%%bq query
-- get geo location for mean location of each stationid
SELECT
   *,
   ST_GeogPoint(lon, lat) AS g
FROM
   `bergqvist-sandbox.bike_demo.hist` 
where not name LIKE "test%"
limit 100
;

stationId,name,timestamp,lon,lat,spacesAvailable,bikesAvailable,g
94,Laajalahden aukio,2017-08-17 18:04:01,24.8759851,60.1983016,2,24,POINT(24.8759851 60.1983016)
93,Torpanranta,2017-05-10 10:15:01,24.8713963,60.1948358,8,8,POINT(24.8713963 60.1948358)
587,Hauenkallio,2018-07-29 07:34:01,24.767831,60.162477,5,7,POINT(24.767831 60.162477)
106,Korppaanmäentie,2017-08-09 10:50:01,24.8989297,60.2034735,6,10,POINT(24.8989297 60.2034735)
78,Messeniuksenkatu,2018-05-31 06:46:01,24.9180102,60.1878352,19,8,POINT(24.9180102 60.1878352)
54,Gyldenintie,2018-06-13 08:25:01,24.8766658,60.1583842,13,17,POINT(24.8766658 60.1583842)
607,Nuottaniementie,2018-05-18 20:45:01,24.741546,60.147921,2,12,POINT(24.741546 60.147921)
749,Vallikatu,2018-07-10 10:09:01,24.813871,60.228463,1,0,POINT(24.813871 60.228463)
49,Annankatu,2018-07-23 07:09:01,24.9417708421,60.1634589429,14,1,POINT(24.94177084206 60.16345894291)
559,Pohjankulma,2018-06-07 02:02:01,24.794014,60.18288,3,28,POINT(24.794014 60.18288)


In [15]:
%%bq query
-- ensure multiply-named stations have not moved too much
SELECT
    name,
    ((max(lat) - min(lat))/ avg(lat))*1000 as milli_degree_mad_lat,
    ((max(lon) - min(lon)) / avg(lon))*1000 as milli_degree_mad_lon
FROM
   `bergqvist-sandbox.bike_demo.hist` 
WHERE stationid in (1, 47, 55, 100) and not name LIKE "test%"
group by name
;


name,milli_degree_mad_lat,milli_degree_mad_lon
Ulvilanpuisto,0.0112878372288,0.106865332241
Kaivopuisto,0.0,0.0
Lahnalahdentie,0.0,0.0
Kauppatori,0.0,0.0
Teljäntie,0.0,0.0
Vanha Kauppahalli,0.0,0.0
Puistokaari,0.0,0.0


In [17]:
%%bq query
-- see what latest date is at each location to check for overlap
SELECT
    name,
    stationid,
    extract(date from TIMESTAMP_SECONDS(min(unix_seconds(`timestamp`))) at time zone "Europe/Helsinki") AS start_date,
    extract(date from TIMESTAMP_SECONDS(max(unix_seconds(`timestamp`))) at time zone "Europe/Helsinki") AS end_date
FROM
   `bergqvist-sandbox.bike_demo.hist` 
WHERE stationid in (1, 47, 55, 100) and not name LIKE "test%"
group by stationid, name
order by stationid, min(unix_seconds(`timestamp`))
;

name,stationid,start_date,end_date
Kaivopuisto,1,2017-05-01,2018-08-03
Kauppatori,47,2017-05-01,2018-03-10
Vanha Kauppahalli,47,2018-03-19,2018-08-03
Lahnalahdentie,55,2017-05-01,2018-03-19
Puistokaari,55,2018-03-19,2018-08-03
Ulvilanpuisto,100,2017-05-01,2018-03-19
Teljäntie,100,2018-03-19,2018-08-03
